In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, \
    MultiThreadSLIM_SLIMElasticNetRecommender

slim = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)

item = ItemKNNCFRecommender(URM_train)

slim.fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668, workers = 7)
item.fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

SLIMElasticNetRecommender: URM Detected 451 ( 3.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 356 ( 1.6%) items with no interactions.
ItemKNNCFRecommender: URM Detected 451 ( 3.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 356 ( 1.6%) items with no interactions.


100%|█████████▉| 22160/22222 [02:03<00:00, 76.40it/s] 

Similarity column 22222 (100.0%), 8057.63 column/sec. Elapsed time 2.76 sec
EvaluatorHoldout: Ignoring 2649 (21.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [5]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
import optuna as op
model = ItemKNNCustomSimilarityRecommender

def objective(trial):
    alpha = trial.suggest_float("alpha", 5e-4, 1)

    new_similarity = (1 - alpha) * slim.W_sparse + alpha * item.W_sparse
    recommender = model(URM_train)
    recommender.fit(W_sparse=new_similarity)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [6]:
best_params = {
}
study_name = "cust-sim-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=100)

[I 2023-12-09 00:57:53,034] Using an existing study with name 'cust-sim-study' instead of creating a new one.


ItemKNNCustomSimilarityRecommender: URM Detected 451 ( 3.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 356 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 9989 (100.0%) in 6.83 sec. Users per second: 1462


[I 2023-12-09 00:57:59,967] Trial 1 finished with value: 0.02582133600929286 and parameters: {'alpha': 0.7050715978800534}. Best is trial 1 with value: 0.02582133600929286.


ItemKNNCustomSimilarityRecommender: URM Detected 451 ( 3.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 356 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 9989 (100.0%) in 7.74 sec. Users per second: 1291


[I 2023-12-09 00:58:07,837] Trial 2 finished with value: 0.029573395814761653 and parameters: {'alpha': 0.28121495054058593}. Best is trial 2 with value: 0.029573395814761653.


...

[I 2023-12-09 01:09:30,239] Trial 98 finished with value: 0.030061551834001776 and parameters: {'alpha': 0.19173256822528087}. Best is trial 69 with value: 0.03047689760323686.


ItemKNNCustomSimilarityRecommender: URM Detected 451 ( 3.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 356 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 9989 (100.0%) in 7.13 sec. Users per second: 1401


[I 2023-12-09 01:09:37,484] Trial 99 finished with value: 0.03046138450708502 and parameters: {'alpha': 0.09779989946234341}. Best is trial 69 with value: 0.03047689760323686.


ItemKNNCustomSimilarityRecommender: URM Detected 451 ( 3.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 356 ( 1.6%) items with no interactions.
EvaluatorHoldout: Processed 9989 (100.0%) in 6.85 sec. Users per second: 1459


[I 2023-12-09 01:09:44,459] Trial 100 finished with value: 0.029966776946705086 and parameters: {'alpha': 0.22020710371900423}. Best is trial 69 with value: 0.03047689760323686.


In [7]:
study.best_params

{'alpha': 0.10204960939360941}

In [10]:
new_similarity = (1 - 0.10204960939360941) * slim.W_sparse + 0.10204960939360941 * item.W_sparse
recommender = model(URM_train)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
recommender.fit(W_sparse=new_similarity)
evaluator_test.evaluateRecommender(recommender)

ItemKNNCustomSimilarityRecommender: URM Detected 451 ( 3.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 356 ( 1.6%) items with no interactions.
EvaluatorHoldout: Ignoring 2137 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10501 (100.0%) in 6.86 sec. Users per second: 1532


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.078231                 0.137502  0.117368  0.037575    0.066342   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.229908  0.123998  0.093884  0.466908      0.292136  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.830907          0.387957    0.830907        0.04454   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10            9.767962                    0.99654             0.129436   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(slim)

EvaluatorHoldout: Ignoring 2137 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10501 (100.0%) in 6.83 sec. Users per second: 1538


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.077831                 0.135528  0.115191  0.037798    0.066234   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.231465  0.123572  0.092895  0.464146      0.293879  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.830907          0.385662    0.830907       0.050025   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.028651                   0.997428             0.145375   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [ ]:
## MAP_MIN_DEN = .0822900 ## MAP = 0.0483066 the best 0.139

In [ ]:
## MAP_MIN_DEN = .0845288 ## MAP = 0.0495724 second 0.135

In [ ]:
## MAP_MIN_DEN = .0846506 ## MAP = 0.0496060 third 